# Model Comparison: Baseline vs Light Augmentation vs Hard Augmentation

Notebook này so sánh 3 chiến lược training:
- **Baseline**: Không áp dụng preprocessing và augmentation
- **Light Augmented**: Augmentation nhẹ (histogram_eq, gaussian_blur, rotation ±10°, hsv_v=0.4, scale=0.5, translate=0.1, fliplr=0.5)
- **Hard Augmented**: Augmentation mạnh (thêm shear, perspective, mosaic, mixup, copy_paste)

## Section 1: Setup and Imports

In [1]:
# !uv pip install -q ultralytics matplotlib seaborn pandas numpy pillow

In [2]:
import os
from pathlib import Path
import yaml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Imports successful")

ModuleNotFoundError: No module named 'seaborn'

## Section 2: Load Model Weights

In [ ]:
# Define model paths
models_dir = Path('../models')

model_configs = {
    'Baseline': {
        'path': models_dir / 'baseline.pt',
        'description': 'Không preprocessing, không augmentation',
        'color': '#FF6B6B'
    },
    'Light Augmented': {
        'path': models_dir / 'light_augmented.pt',
        'description': 'Histogram EQ + Gaussian blur + Rotation ±10° + HSV + Scale + Translate + Flip',
        'color': '#4ECDC4'
    },
    'Hard Augmented': {
        'path': models_dir / 'hard_augmented.pt',
        'description': 'Light + Shear + Perspective + Mosaic + Mixup + Copy-Paste',
        'color': '#95E1D3'
    }
}

# Verify all models exist
print("Verifying Model Weights")
print("=" * 80)
for name, config in model_configs.items():
    path = config['path']
    if path.exists():
        size_mb = path.stat().st_size / (1024*1024)
        print(f"✓ {name:20s}: {path} ({size_mb:.2f} MB)")
    else:
        print(f"✗ {name:20s}: NOT FOUND at {path}")
        raise FileNotFoundError(f"Model not found: {path}")

print("\n✓ All model weights found!")
print("=" * 80)

Verifying Model Weights
✓ Baseline            : ../models/baseline.pt (18.36 MB)
✓ Light Augmented     : ../models/light_augmented.pt (18.36 MB)
✓ Hard Augmented      : ../models/hard_augmented.pt (18.37 MB)

✓ All model weights found!


## Section 3: Verify Test Data

In [ ]:
# Define data paths for each model
# Baseline: evaluate on baseline_2classes (raw, no preprocessing)
baseline_data_yaml = Path('../data/baseline_2classes/data.yaml')

# Light & Hard Augmented: evaluate on preprocessed_2classes_aug_non_negative (with preprocessing)
preprocessed_data_yaml = Path('../data/preprocessed_2classes_aug_non_negative/data.yaml')

print("Verifying Test Datasets")
print("=" * 80)

# Verify baseline dataset
if not baseline_data_yaml.exists():
    print(f"ERROR: baseline data.yaml not found at {baseline_data_yaml}")
    raise FileNotFoundError(f"Baseline data.yaml not found")

print(f"✓ Baseline data.yaml found: {baseline_data_yaml}")

# Load baseline data config
with open(baseline_data_yaml, 'r') as f:
    baseline_data_config = yaml.safe_load(f)

print(f"  Classes: {baseline_data_config['nc']} | Names: {baseline_data_config['names']}")

# Count baseline test images
baseline_test_images = Path(baseline_data_yaml.parent) / 'test' / 'images'
baseline_test_count = len(list(baseline_test_images.glob('*.png'))) + len(list(baseline_test_images.glob('*.jpg')))
print(f"  Test images: {baseline_test_count:,}")

print()

# Verify preprocessed dataset
if not preprocessed_data_yaml.exists():
    print(f"ERROR: preprocessed data.yaml not found at {preprocessed_data_yaml}")
    raise FileNotFoundError(f"Preprocessed data.yaml not found")

print(f"✓ Preprocessed data.yaml found: {preprocessed_data_yaml}")

# Load preprocessed data config
with open(preprocessed_data_yaml, 'r') as f:
    preprocessed_data_config = yaml.safe_load(f)

print(f"  Classes: {preprocessed_data_config['nc']} | Names: {preprocessed_data_config['names']}")

# Count preprocessed test images
preprocessed_test_images = Path(preprocessed_data_yaml.parent) / 'test' / 'images'
preprocessed_test_count = len(list(preprocessed_test_images.glob('*.png'))) + len(list(preprocessed_test_images.glob('*.jpg')))
print(f"  Test images: {preprocessed_test_count:,}")

if baseline_test_count == 0 or preprocessed_test_count == 0:
    print("\nERROR: No test images found!")
    raise ValueError("No test images found in one or both datasets")

print("\n✓ Both datasets verified successfully!")
print("=" * 80)

Verifying Test Datasets
✓ Baseline data.yaml found: ../data/baseline_2classes/data.yaml
  Classes: 2 | Names: ['Aortic enlargement', 'Cardiomegaly']
  Test images: 328

✓ Preprocessed data.yaml found: ../data/preprocessed_2classes_aug_non_negative/data.yaml
  Classes: 2 | Names: ['Aortic enlargement', 'Cardiomegaly']
  Test images: 328

✓ Both datasets verified successfully!


## Section 4: Evaluate Models on Test Set

In [ ]:
# Evaluate all models
results = {}

print("Evaluating Models on Respective Test Datasets")
print("=" * 80)

for name, config in model_configs.items():
    print(f"\nEvaluating: {name}")
    print(f"Description: {config['description']}")
    print("-" * 80)
    
    # Select correct dataset for each model
    if name == 'Baseline':
        eval_data_yaml = str(baseline_data_yaml)
        dataset_name = "baseline_2classes (raw, no preprocessing)"
    else:
        eval_data_yaml = str(preprocessed_data_yaml)
        dataset_name = "preprocessed_2classes_aug_non_negative (with histogram_eq)"
    
    print(f"Dataset: {dataset_name}")
    
    # Load model
    model = YOLO(str(config['path']))
    
    # Run validation on test set
    metrics = model.val(data=eval_data_yaml, split='test', verbose=False)
    
    # Extract metrics
    results_dict = metrics.results_dict
    results[name] = {
        'mAP50': results_dict.get('metrics/mAP50(B)', 0),
        'mAP50-95': results_dict.get('metrics/mAP50-95(B)', 0),
        'Precision': results_dict.get('metrics/precision(B)', 0),
        'Recall': results_dict.get('metrics/recall(B)', 0),
        'color': config['color'],
        'dataset': dataset_name
    }
    
    print(f"  mAP50:       {results[name]['mAP50']:.4f}")
    print(f"  mAP50-95:    {results[name]['mAP50-95']:.4f}")
    print(f"  Precision:   {results[name]['Precision']:.4f}")
    print(f"  Recall:      {results[name]['Recall']:.4f}")

print("\n" + "=" * 80)
print("✓ All models evaluated successfully!")
print("=" * 80)

Evaluating Models on Respective Test Datasets

Evaluating: Baseline
Description: Không preprocessing, không augmentation
--------------------------------------------------------------------------------
Dataset: baseline_2classes (raw, no preprocessing)
Ultralytics 8.3.226 🚀 Python-3.12.3 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 11906MiB)
Ultralytics 8.3.226 🚀 Python-3.12.3 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 11906MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3971.4±1249.2 MB/s, size: 64.6 KB)
val: Scanning /home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray/data/baseline_2classes/test/labels.cache... 328 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 328/328 980.6Kit/s 0.0s
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3971.4±1249.2 

## Section 5: Comparison Metrics Table

In [ ]:
# Create comparison dataframe
df_results = pd.DataFrame(results).T
dataset_info = df_results[['dataset']]
df_results = df_results.drop(columns=['color', 'dataset'])

print("\nComparison Metrics Table")
print("=" * 80)
print("\nEvaluation Datasets per Model:")
print("-" * 80)
for name, dataset in dataset_info['dataset'].items():
    print(f"  {name:20s}: {dataset}")

print("\nMetrics:")
print("-" * 80)
print(df_results.to_string())
print("=" * 80)

# Calculate improvements
print("\n\nImprovement Analysis (vs Baseline)")
print("=" * 80)

baseline_metrics = results['Baseline']

for name in ['Light Augmented', 'Hard Augmented']:
    print(f"\n{name}:")
    print(f"  (Evaluated on: {results[name]['dataset']})")
    print(f"  Note: So sánh này có thể không hoàn toàn công bằng do khác dataset")
    for metric in ['mAP50', 'mAP50-95', 'Precision', 'Recall']:
        baseline_val = baseline_metrics[metric]
        current_val = results[name][metric]
        
        if baseline_val > 0:
            improvement = ((current_val - baseline_val) / baseline_val) * 100
            symbol = '↑' if improvement > 0 else '↓'
            print(f"  {metric:15s}: {current_val:.4f} ({symbol} {abs(improvement):+.2f}%)")
        else:
            print(f"  {metric:15s}: {current_val:.4f} (N/A - baseline is 0)")

print("\n" + "=" * 80)


Comparison Metrics Table

Evaluation Datasets per Model:
--------------------------------------------------------------------------------
  Baseline            : baseline_2classes (raw, no preprocessing)
  Light Augmented     : preprocessed_2classes_aug_non_negative (with histogram_eq)
  Hard Augmented      : preprocessed_2classes_aug_non_negative (with histogram_eq)

Metrics:
--------------------------------------------------------------------------------
                    mAP50  mAP50-95 Precision    Recall
Baseline         0.903369  0.456683  0.842646  0.882642
Light Augmented  0.931034  0.491081  0.851754  0.928145
Hard Augmented    0.94179  0.502943  0.911658  0.905483


Improvement Analysis (vs Baseline)

Light Augmented:
  (Evaluated on: preprocessed_2classes_aug_non_negative (with histogram_eq))
  Note: So sánh này có thể không hoàn toàn công bằng do khác dataset
  mAP50          : 0.9310 (↑ +3.06%)
  mAP50-95       : 0.4911 (↑ +7.53%)
  Precision      : 0.8518 (↑ +1.08%)
  

## Section 6: Visualization - Bar Charts

In [ ]:
# Create bar chart comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Model Comparison: Baseline vs Light Augmented vs Hard Augmented', 
             fontsize=16, fontweight='bold', y=0.995)

metrics_to_plot = ['mAP50', 'mAP50-95', 'Precision', 'Recall']
axes_flat = axes.flatten()

for idx, metric in enumerate(metrics_to_plot):
    ax = axes_flat[idx]
    
    # Extract data
    model_names = list(results.keys())
    values = [results[name][metric] for name in model_names]
    colors = [results[name]['color'] for name in model_names]
    
    # Create bar chart
    bars = ax.bar(model_names, values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Styling
    ax.set_title(f'{metric}', fontsize=14, fontweight='bold', pad=10)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_ylim(0, max(values) * 1.15)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)
    
    # Rotate x-axis labels
    ax.set_xticklabels(model_names, rotation=15, ha='right')

plt.tight_layout()
plt.savefig('../docs/model_comparison_bars.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Bar chart saved to: docs/model_comparison_bars.png")

/tmp/ipykernel_110125/3901160943.py:35: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(model_names, rotation=15, ha='right')
/tmp/ipykernel_110125/3901160943.py:35: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(model_names, rotation=15, ha='right')
/tmp/ipykernel_110125/3901160943.py:35: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(model_names, rotation=15, ha='right')
/tmp/ipykernel_110125/3901160943.py:35: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(model_names, rotation=15, ha='right')


<Figure size 1600x1200 with 4 Axes>

✓ Bar chart saved to: docs/model_comparison_bars.png


## Section 7: Visualization - Radar Chart

In [ ]:
# Create radar chart
from math import pi

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(projection='polar'))

# Define metrics for radar chart
categories = ['mAP50', 'mAP50-95', 'Precision', 'Recall']
N = len(categories)

# Compute angle for each axis
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]  # Complete the circle

# Plot each model
for name in results.keys():
    values = [results[name][metric] for metric in categories]
    values += values[:1]  # Complete the circle
    
    ax.plot(angles, values, 'o-', linewidth=2.5, label=name, 
            color=results[name]['color'], markersize=8)
    ax.fill(angles, values, alpha=0.15, color=results[name]['color'])

# Fix axis to go in the right order
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=12, fontweight='bold')

# Set y-axis limits
ax.set_ylim(0, 1.0)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=10)

# Add legend
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)

# Add title
plt.title('Model Performance Radar Chart\nBaseline vs Light Augmented vs Hard Augmented',
          fontsize=14, fontweight='bold', pad=30)

# Add grid
ax.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('../docs/model_comparison_radar.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Radar chart saved to: docs/model_comparison_radar.png")

<Figure size 1200x1200 with 1 Axes>

✓ Radar chart saved to: docs/model_comparison_radar.png


## Section 8: Improvement Heatmap

In [ ]:
# Calculate percentage improvements relative to baseline
improvement_data = []

baseline_metrics = results['Baseline']

for name in ['Baseline', 'Light Augmented', 'Hard Augmented']:
    row = []
    for metric in ['mAP50', 'mAP50-95', 'Precision', 'Recall']:
        baseline_val = baseline_metrics[metric]
        current_val = results[name][metric]
        
        if baseline_val > 0:
            improvement = ((current_val - baseline_val) / baseline_val) * 100
        else:
            improvement = 0
        
        row.append(improvement)
    
    improvement_data.append(row)

# Create dataframe
df_improvement = pd.DataFrame(
    improvement_data,
    index=['Baseline', 'Light Augmented', 'Hard Augmented'],
    columns=['mAP50', 'mAP50-95', 'Precision', 'Recall']
)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 6))

sns.heatmap(df_improvement, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
            cbar_kws={'label': 'Improvement (%)'}, linewidths=2, linecolor='white',
            ax=ax, vmin=-10, vmax=50)

ax.set_title('Improvement Heatmap (% change vs Baseline)', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Metrics', fontsize=12, fontweight='bold')
ax.set_ylabel('Model', fontsize=12, fontweight='bold')

# Rotate labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='center')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, ha='right')

plt.tight_layout()
plt.savefig('../docs/model_comparison_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Heatmap saved to: docs/model_comparison_heatmap.png")

<Figure size 1000x600 with 2 Axes>

✓ Heatmap saved to: docs/model_comparison_heatmap.png


## Section 9: Summary Statistics

In [ ]:
print("\n" + "=" * 80)
print("SUMMARY STATISTICS")
print("=" * 80)

print("\n⚠️  IMPORTANT NOTES:")
print("-" * 80)
print("  • Baseline được evaluate trên: baseline_2classes (raw, chưa preprocessing)")
print("  • Light & Hard Augmented được evaluate trên: preprocessed_2classes_aug_non_negative")
print("  • Do vậy, so sánh improvement có thể không hoàn toàn công bằng")
print("  • Để so sánh công bằng, nên cũng evaluate Light/Hard trên baseline_2classes")

# Find best model for each metric
print("\nBest Model per Metric:")
print("-" * 80)

for metric in ['mAP50', 'mAP50-95', 'Precision', 'Recall']:
    best_model = max(results.keys(), key=lambda x: results[x][metric])
    best_value = results[best_model][metric]
    dataset = results[best_model]['dataset']
    print(f"  {metric:15s}: {best_model:20s} ({best_value:.4f})")
    print(f"                   → Evaluated on: {dataset}")

# Calculate average improvements
print("\nAverage Improvement vs Baseline:")
print("-" * 80)

for name in ['Light Augmented', 'Hard Augmented']:
    improvements = []
    for metric in ['mAP50', 'mAP50-95', 'Precision', 'Recall']:
        baseline_val = baseline_metrics[metric]
        current_val = results[name][metric]
        if baseline_val > 0:
            improvement = ((current_val - baseline_val) / baseline_val) * 100
            improvements.append(improvement)
    
    avg_improvement = np.mean(improvements)
    symbol = '↑' if avg_improvement > 0 else '↓'
    print(f"  {name:20s}: {symbol} {abs(avg_improvement):.2f}%")
    print(f"  (⚠️  So sánh trên khác dataset, có thể không chính xác)")

# Recommendations
print("\n" + "=" * 80)
print("RECOMMENDATIONS")
print("=" * 80)

# Find overall best model (based on mAP50-95)
best_overall = max(results.keys(), key=lambda x: results[x]['mAP50-95'])
print(f"\n✓ Best Model (mAP50-95): {best_overall}")
print(f"  Score: {results[best_overall]['mAP50-95']:.4f}")
print(f"  Dataset: {results[best_overall]['dataset']}")
print(f"  Description: {model_configs[best_overall]['description']}")

# Analysis
print("\nKey Findings:")
print("-" * 80)
print("  1. Baseline (raw dataset) provides baseline performance")
print("  2. Light Augmented (preprocessed) helps with histogram equalization")
print("  3. Hard Augmented (preprocessed) thêm aggressive augmentation")
print("\n  ⚠️  IMPORTANT:")
print("  Để so sánh công bằng, hãy tạo một evaluation trên cùng dataset!")
print("  → Evaluate Light/Hard Augmented trên baseline_2classes để thấy")
print("    hiệu ứng thực sự của augmentation (độc lập với preprocessing)")

print("\n" + "=" * 80)


SUMMARY STATISTICS

⚠️  IMPORTANT NOTES:
--------------------------------------------------------------------------------
  • Baseline được evaluate trên: baseline_2classes (raw, chưa preprocessing)
  • Light & Hard Augmented được evaluate trên: preprocessed_2classes_aug_non_negative
  • Do vậy, so sánh improvement có thể không hoàn toàn công bằng
  • Để so sánh công bằng, nên cũng evaluate Light/Hard trên baseline_2classes

Best Model per Metric:
--------------------------------------------------------------------------------
  mAP50          : Hard Augmented       (0.9418)
                   → Evaluated on: preprocessed_2classes_aug_non_negative (with histogram_eq)
  mAP50-95       : Hard Augmented       (0.5029)
                   → Evaluated on: preprocessed_2classes_aug_non_negative (with histogram_eq)
  Precision      : Hard Augmented       (0.9117)
                   → Evaluated on: preprocessed_2classes_aug_non_negative (with histogram_eq)
  Recall         : Light Augmented   

## Section 11: Fair Comparison - All Models on Baseline Dataset (Optional)

In [ ]:
# Fair comparison: Evaluate all models on the same baseline dataset
print("\n" + "=" * 80)
print("FAIR COMPARISON: All Models on Baseline Dataset")
print("=" * 80)

results_fair = {}

print("\nEvaluating all models on baseline_2classes (raw, no preprocessing)")
print("-" * 80)

for name, config in model_configs.items():
    print(f"\nEvaluating: {name}")
    
    # Load model
    model = YOLO(str(config['path']))
    
    # Run validation on baseline test set
    metrics = model.val(data=str(baseline_data_yaml), split='test', verbose=False)
    
    # Extract metrics
    results_dict = metrics.results_dict
    results_fair[name] = {
        'mAP50': results_dict.get('metrics/mAP50(B)', 0),
        'mAP50-95': results_dict.get('metrics/mAP50-95(B)', 0),
        'Precision': results_dict.get('metrics/precision(B)', 0),
        'Recall': results_dict.get('metrics/recall(B)', 0),
        'color': config['color']
    }
    
    print(f"  mAP50:       {results_fair[name]['mAP50']:.4f}")
    print(f"  mAP50-95:    {results_fair[name]['mAP50-95']:.4f}")
    print(f"  Precision:   {results_fair[name]['Precision']:.4f}")
    print(f"  Recall:      {results_fair[name]['Recall']:.4f}")

print("\n" + "=" * 80)

# Create comparison dataframe for fair comparison
df_results_fair = pd.DataFrame(results_fair).T
df_results_fair = df_results_fair.drop(columns=['color'])

print("\nFair Comparison Metrics Table (All on baseline_2classes)")
print("=" * 80)
print(df_results_fair.to_string())
print("=" * 80)

# Calculate improvements for fair comparison
print("\n\nFair Improvement Analysis (vs Baseline - same dataset)")
print("=" * 80)

baseline_metrics_fair = results_fair['Baseline']

for name in ['Light Augmented', 'Hard Augmented']:
    print(f"\n{name}:")
    for metric in ['mAP50', 'mAP50-95', 'Precision', 'Recall']:
        baseline_val = baseline_metrics_fair[metric]
        current_val = results_fair[name][metric]
        
        if baseline_val > 0:
            improvement = ((current_val - baseline_val) / baseline_val) * 100
            symbol = '↑' if improvement > 0 else '↓'
            print(f"  {metric:15s}: {current_val:.4f} ({symbol} {abs(improvement):+.2f}%)")
        else:
            print(f"  {metric:15s}: {current_val:.4f} (N/A - baseline is 0)")

print("\n" + "=" * 80)
print("✓ Fair comparison complete - hiệu ứng thực sự của augmentation trên raw data")
print("=" * 80)


FAIR COMPARISON: All Models on Baseline Dataset

Evaluating all models on baseline_2classes (raw, no preprocessing)
--------------------------------------------------------------------------------

Evaluating: Baseline
Ultralytics 8.3.226 🚀 Python-3.12.3 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 11906MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4466.6±1256.2 MB/s, size: 71.9 KB)
val: Scanning /home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray/data/baseline_2classes/test/labels.cache... 328 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 328/328 846.1Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 21/21 3.3it/s 6.4s0.3s
                   all        328        519      0.843      0.883      0.903      0.457
Speed: 2.0ms preprocess, 14.7ms inference, 0.0ms loss, 0.3

## Section 12: Fair Comparison - Visualizations

In [ ]:
# Create bar chart comparison for fair results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Fair Model Comparison (All on Baseline Dataset)', 
             fontsize=16, fontweight='bold', y=0.995)

metrics_to_plot = ['mAP50', 'mAP50-95', 'Precision', 'Recall']
axes_flat = axes.flatten()

for idx, metric in enumerate(metrics_to_plot):
    ax = axes_flat[idx]
    
    # Extract data
    model_names = list(results_fair.keys())
    values = [results_fair[name][metric] for name in model_names]
    colors = [results_fair[name]['color'] for name in model_names]
    
    # Create bar chart
    bars = ax.bar(model_names, values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Styling
    ax.set_title(f'{metric}', fontsize=14, fontweight='bold', pad=10)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_ylim(0, max(values) * 1.15)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)
    
    # Rotate x-axis labels
    ax.set_xticklabels(model_names, rotation=15, ha='right')

plt.tight_layout()
plt.savefig('../docs/model_comparison_bars_fair.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Fair comparison bar chart saved to: docs/model_comparison_bars_fair.png")

In [ ]:
# Create radar chart for fair comparison
from math import pi

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(projection='polar'))

# Define metrics for radar chart
categories = ['mAP50', 'mAP50-95', 'Precision', 'Recall']
N = len(categories)

# Compute angle for each axis
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]  # Complete the circle

# Plot each model
for name in results_fair.keys():
    values = [results_fair[name][metric] for metric in categories]
    values += values[:1]  # Complete the circle
    
    ax.plot(angles, values, 'o-', linewidth=2.5, label=name, 
            color=results_fair[name]['color'], markersize=8)
    ax.fill(angles, values, alpha=0.15, color=results_fair[name]['color'])

# Fix axis to go in the right order
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=12, fontweight='bold')

# Set y-axis limits
ax.set_ylim(0, 1.0)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=10)

# Add legend
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)

# Add title
plt.title('Fair Model Performance Radar Chart (All on Baseline Dataset)',
          fontsize=14, fontweight='bold', pad=30)

# Add grid
ax.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('../docs/model_comparison_radar_fair.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Fair comparison radar chart saved to: docs/model_comparison_radar_fair.png")

In [ ]:
# Calculate percentage improvements for fair comparison
improvement_data_fair = []

baseline_metrics_fair = results_fair['Baseline']

for name in ['Baseline', 'Light Augmented', 'Hard Augmented']:
    row = []
    for metric in ['mAP50', 'mAP50-95', 'Precision', 'Recall']:
        baseline_val = baseline_metrics_fair[metric]
        current_val = results_fair[name][metric]
        
        if baseline_val > 0:
            improvement = ((current_val - baseline_val) / baseline_val) * 100
        else:
            improvement = 0
        
        row.append(improvement)
    
    improvement_data_fair.append(row)

# Create dataframe
df_improvement_fair = pd.DataFrame(
    improvement_data_fair,
    index=['Baseline', 'Light Augmented', 'Hard Augmented'],
    columns=['mAP50', 'mAP50-95', 'Precision', 'Recall']
)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 6))

sns.heatmap(df_improvement_fair, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
            cbar_kws={'label': 'Improvement (%)'}, linewidths=2, linecolor='white',
            ax=ax, vmin=-10, vmax=10)

ax.set_title('Fair Improvement Heatmap (% change vs Baseline on same dataset)', 
             fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Metrics', fontsize=12, fontweight='bold')
ax.set_ylabel('Model', fontsize=12, fontweight='bold')

# Rotate labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='center')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, ha='right')

plt.tight_layout()
plt.savefig('../docs/model_comparison_heatmap_fair.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Fair comparison heatmap saved to: docs/model_comparison_heatmap_fair.png")

In [ ]:
# Export results to CSV
output_dir = Path('../docs')
output_dir.mkdir(exist_ok=True)

# Save comparison metrics
df_results.to_csv(output_dir / 'model_comparison_metrics.csv')
print(f"✓ Metrics saved to: {output_dir / 'model_comparison_metrics.csv'}")

# Save improvement percentages
df_improvement.to_csv(output_dir / 'model_comparison_improvements.csv')
print(f"✓ Improvements saved to: {output_dir / 'model_comparison_improvements.csv'}")

print("\n✓ All results exported successfully!")

✓ Metrics saved to: ../docs/model_comparison_metrics.csv
✓ Improvements saved to: ../docs/model_comparison_improvements.csv

✓ All results exported successfully!


## Conclusion

Notebook này đã:
1. ✓ Load 3 model weights (baseline, light_augmented, hard_augmented)
2. ✓ Evaluate trên test set
3. ✓ So sánh metrics (mAP50, mAP50-95, Precision, Recall)
4. ✓ Tạo 3 loại biểu đồ: Bar chart, Radar chart, Heatmap
5. ✓ Phân tích improvement và đưa ra recommendations
6. ✓ Thực hiện so sánh công bằng (Fair Comparison) trên cùng một dataset và trực quan hóa kết quả.
7. ✓ Export kết quả ra CSV files

**Các file output:**
- `docs/model_comparison_bars.png` - Bar chart comparison
- `docs/model_comparison_radar.png` - Radar chart comparison  
- `docs/model_comparison_heatmap.png` - Improvement heatmap
- `docs/model_comparison_bars_fair.png` - Fair comparison bar chart
- `docs/model_comparison_radar_fair.png` - Fair comparison radar chart
- `docs/model_comparison_heatmap_fair.png` - Fair comparison heatmap
- `docs/model_comparison_metrics.csv` - Raw metrics
- `docs/model_comparison_improvements.csv` - Improvement percentages